# Experiment

## Install Packages

In [141]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [142]:
!pip install --quiet seaborn numpy pandas river scikit-learn lightgbm matplotlib deepchecks evidently menelaus tensorflow-data-validation

In [143]:
import warnings

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from lightgbm import LGBMClassifier
import drift_detector_with_labels, deepcheck_detectors, evidently_ai_detectors, drift, drift_detector_multivariate_hdddm, drift_detector_multivariate_md3, drift_detector_multivariate_ollindda, tensorflow_detectors

warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

# Load Dataset & EDA

In [144]:
red_wine_dataset = pd.read_csv(
    "https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv", sep=";")
white_wine_dataset = pd.read_csv(
    "https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv", sep=";")

In [145]:
red_wine_dataset['wine_type'] = "red"
white_wine_dataset['wine_type'] = "white"

In [146]:
wine_dataset = pd.concat([red_wine_dataset, white_wine_dataset], axis=0)
wine_dataset["wine_type"] = np.where(wine_dataset["wine_type"] == "red", 1, 0)
wine_dataset.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,wine_type
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,1
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5,1
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5,1
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,1


In [147]:
conditions = [
    (wine_dataset["quality"] <= 3),
    (wine_dataset.quality > 3) & (wine_dataset.quality <= 6),
    (wine_dataset["quality"] > 6)
]

values = [0, 1, 2]

wine_dataset['quality'] = np.select(conditions, values)

wine_dataset['quality'].value_counts()

1    5190
2    1277
0      30
Name: quality, dtype: int64

In [148]:
wine_dataset_train, wine_dataset_test = train_test_split(wine_dataset, test_size=0.4, shuffle=True)
X_train = wine_dataset_train.loc[:, wine_dataset_train.columns != "quality"]
y_train = wine_dataset_train["quality"]
X_test = wine_dataset_test.loc[:, wine_dataset_test.columns != "quality"]
y_test = wine_dataset_test["quality"]

## Fit Light GBM

In [149]:
categorical_features_names = ["wine_type"]
features_names = X_train.columns
cat_features_index = [index for index, feature_name in enumerate(features_names) if
                      feature_name in categorical_features_names]

In [150]:
model_params = {
    'learning_rate': 0.1,
    'max_depth': None,
    'n_estimators': 500,
    'min_child_samples': 10,
    'categorical_feature': cat_features_index,
    'n_jobs': 1,
    'random_state': 1234,
}
lgbm_model = LGBMClassifier(**model_params)
lgbm_model.fit(X_train, y_train)

LGBMClassifier(categorical_feature=[11], max_depth=None, min_child_samples=10,
               n_estimators=500, n_jobs=1, random_state=1234)

In [151]:
y_pred = lgbm_model.predict(X_test)
accuracy_score(y_pred, y_test)

0.8657175836860331

## Real Concept Drift

### Induce the drift

In [152]:
wine_dataset_corrupted_concept_drift = drift.drift_generator_concept_drift(data=wine_dataset_train,
                                                                           label_col="quality",
                                                                           label_value=0,
                                                                           column_name="alcohol",
                                                                           value=12,
                                                                           action="greater")

In [153]:
wine_dataset_corrupted_concept_drift["alcohol"].corr(wine_dataset_corrupted_concept_drift["quality"])

-0.3991068806388971

In [154]:
wine_dataset_test["alcohol"].corr(wine_dataset_test["quality"])

0.38058870688834934

### Deepcheck

In [155]:
deepcheck_detectors.deepcheck_detect_drift(data_train=wine_dataset_train,
                                           data_to_compare=wine_dataset_corrupted_concept_drift,
                                           label_col="quality",
                                           cat_features=categorical_features_names,
                                           model=lgbm_model,
                                           test_type="feature_drift")

In [156]:
deepcheck_detectors.deepcheck_detect_drift(data_train=wine_dataset_train,
                                           data_to_compare=wine_dataset_corrupted_concept_drift,
                                           label_col="quality",
                                           cat_features=categorical_features_names,
                                           model=lgbm_model,
                                           test_type="prediction_drift")

No drift detected


In [158]:
#deepcheck_detectors.deepcheck_detect_drift(data_train=wine_dataset_train,
#                                           data_to_compare=wine_dataset_corrupted_concept_drift,
#                                           label_col="quality",
#                                           cat_features=categorical_features_names,
#                                           model=lgbm_model,
#                                           test_type="dataset_drift")

### Evidently AI

In [159]:
evidently_ai_detectors.evidently_ai_detect_drift(data_train=wine_dataset_train,
                                                 data_to_compare=wine_dataset_corrupted_concept_drift,
                                                 label_col="quality",
                                                 cat_features=categorical_features_names)

Alarm quality


### Tensorflow

In [160]:
tensorflow_detectors.tensorflow_detect_drift(data_train=wine_dataset_train,
                                             data_to_compare=wine_dataset_corrupted_concept_drift,
                                             label_col="quality")

In [161]:
wine_dataset_corrupted_concept_drift = wine_dataset_corrupted_concept_drift.reset_index(drop=True)
wine_dataset_train = wine_dataset_train.reset_index(drop=True)

### Drift Detector with Labels

In [163]:
data_wine = pd.concat([wine_dataset_train, wine_dataset_corrupted_concept_drift], axis=0)

#### EDDM

In [164]:
drift_detector_with_labels.drift_detector_with_labels_test(data_to_compare=data_wine,
                                                           label_col="quality",
                                                           model=lgbm_model,
                                                           test_name="EDDM")

Change detected at index 4164
Warning detected at index 4362
Change detected at index 4841
Warning detected at index 5075
Change detected at index 5321
Change detected at index 5517
Warning detected at index 5844


#### HDDM_W

In [165]:
drift_detector_with_labels.drift_detector_with_labels_test(data_to_compare=data_wine,
                                                           label_col="quality",
                                                           model=lgbm_model,
                                                           test_name="HDDM_W")

#### ADWIN

In [166]:
drift_detector_with_labels.drift_detector_with_labels_test(data_to_compare=data_wine,
                                                           label_col="quality",
                                                           model=lgbm_model,
                                                           test_name="ADWIN")

Change detected at index 4095
Change detected at index 4159
Change detected at index 4191
Change detected at index 4223
Change detected at index 4319
Change detected at index 4607
Change detected at index 5567


### Drift Detector Multivariate: HDDDM & MD3

#### HDDDM

In [167]:
X_train = wine_dataset_train.loc[:, wine_dataset_train.columns != "quality"]
y_train = wine_dataset_train["quality"]

In [168]:
X_corrupted = wine_dataset_corrupted_concept_drift.loc[:, wine_dataset_corrupted_concept_drift.columns != "quality"]
y_corrupted = wine_dataset_corrupted_concept_drift["quality"]

In [169]:
drift_detector_multivariate_hdddm.hdddm_detect_drift(data_train=X_train,
                                                     data_to_compare=X_corrupted,
                                                     gamma_level=0.05)

None


#### MD3

In [170]:
drift_detector_multivariate_md3.md3_detect_drift(data_train=wine_dataset_train,
                                                 data_to_compare=wine_dataset_corrupted_concept_drift,
                                                 label_col="quality")

#### OLINDDA

In [171]:
drift_detector_multivariate_ollindda.olindda_detect_drift(x_train_data=X_train,
                                                          x_test_data=X_corrupted,
                                                          n_clusters=6)

No drift or novelty detected in batch


False

## Virtual Concept Drift: alcohol increase by 2

### Induce the drift

In [172]:
wine_dataset_corrupted_alcohol = drift.drift_generator_univariate_increase(data=wine_dataset_test,
                                                                           column_name='alcohol',
                                                                           value=2)

### Deepcheck

In [173]:
deepcheck_detectors.deepcheck_detect_drift(data_train=wine_dataset_train,
                                           data_to_compare=wine_dataset_corrupted_alcohol,
                                           label_col="quality",
                                           cat_features=categorical_features_names,
                                           model=lgbm_model,
                                           test_type="feature_drift")

Alarm alcohol


In [174]:
deepcheck_detectors.deepcheck_detect_drift(data_train=wine_dataset_train,
                                           data_to_compare=wine_dataset_corrupted_alcohol,
                                           label_col="quality",
                                           cat_features=categorical_features_names,
                                           model=lgbm_model,
                                           test_type="prediction_drift")

Warning


In [175]:
#deepcheck_detectors.deepcheck_detect_drift(data_train=wine_dataset_train,
#                                           data_to_compare=wine_dataset_corrupted_alcohol,
#                                           label_col="quality",
#                                           cat_features=categorical_features_names,
#                                           model=lgbm_model,
#                                           test_type="dataset_drift")

### Evidently AI

In [176]:
evidently_ai_detectors.evidently_ai_detect_drift(data_train=wine_dataset_train,
                                                 data_to_compare=wine_dataset_corrupted_alcohol,
                                                 label_col="quality",
                                                 cat_features=categorical_features_names)

Alarm alcohol


### Tensorflow

In [177]:
tensorflow_detectors.tensorflow_detect_drift(data_train=wine_dataset_train,
                                             data_to_compare=wine_dataset_corrupted_alcohol,
                                             label_col="quality")

,Anomaly short description,Anomaly long description
Feature name,,
'chlorides',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 0.0295839 (up to six significant digits), above the threshold 0.01."
'pH',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 0.0249625 (up to six significant digits), above the threshold 0.01."
'volatile acidity',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 0.0318468 (up to six significant digits), above the threshold 0.01."
'sulphates',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 0.0382172 (up to six significant digits), above the threshold 0.01."
'fixed acidity',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 0.026148 (up to six significant digits), above the threshold 0.01."
'free sulfur dioxide',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 0.0247023 (up to six significant digits), above the threshold 0.01."
'density',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 0.0690513 (up to six significant digits), above the threshold 0.01."
'citric acid',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 0.0356787 (up to six significant digits), above the threshold 0.01."
'alcohol',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 0.447917 (up to six significant digits), above the threshold 0.01."


### Drift Detectors with Labels

In [178]:
data_wine_alcohol = pd.concat([wine_dataset_train, wine_dataset_corrupted_alcohol], axis=0)

#### EDDM

In [179]:
drift_detector_with_labels.drift_detector_with_labels_test(data_to_compare=data_wine_alcohol,
                                                           label_col="quality",
                                                           model=lgbm_model,
                                                           test_name="EDDM")

Change detected at index 4042
Change detected at index 4198
Change detected at index 4365
Warning detected at index 4615
Change detected at index 4646
Warning detected at index 4793


#### ADWIN

In [180]:
drift_detector_with_labels.drift_detector_with_labels_test(data_to_compare=data_wine_alcohol,
                                                           label_col="quality",
                                                           model=lgbm_model,
                                                           test_name="ADWIN")

Change detected at index 3999
Change detected at index 4031
Change detected at index 4063
Change detected at index 4191
Change detected at index 4223
Change detected at index 4415


#### HDDM_W

In [181]:
drift_detector_with_labels.drift_detector_with_labels_test(data_to_compare=data_wine_alcohol,
                                                           label_col="quality",
                                                           model=lgbm_model,
                                                           test_name="HDDM_W")

Warning detected at index 3957
Change detected at index 3958
Warning detected at index 4199


### Drift Detector Multivariate: HDDDM & MD3
#### HDDDM

In [182]:
X_train = wine_dataset_train.loc[:, wine_dataset_train.columns != "quality"]
X_corrupted = wine_dataset_corrupted_alcohol.loc[:, wine_dataset_corrupted_alcohol.columns != "quality"]
drift_detector_multivariate_hdddm.hdddm_detect_drift(data_train=X_train,
                                                     data_to_compare=X_corrupted,
                                                     gamma_level=0.05)

None


#### MD3

In [183]:
drift_detector_multivariate_md3.md3_detect_drift(data_train=wine_dataset_train,
                                                 data_to_compare=wine_dataset_corrupted_alcohol,
                                                 label_col="quality")

#### OLINDDA

In [184]:
drift_detector_multivariate_ollindda.olindda_detect_drift(x_train_data=X_train,
                                                          x_test_data=X_corrupted,
                                                          n_clusters=6)

No drift or novelty detected in batch


False

## Virtual Concept Drift: decrease the total sulfur dioxide variance by 100

### Induce the drift

In [185]:
sigma = (wine_dataset_test["total sulfur dioxide"].var() - 100)
mu = wine_dataset_test["total sulfur dioxide"].mean()
wine_dataset_corrupted_dioxide = drift.drift_generator_univariate_change_to_normal(data=wine_dataset_test,
                                                                                   column_name="total sulfur dioxide",
                                                                                   seed=202,
                                                                                   mu=mu,
                                                                                   sigma=sigma)

### Deepcheck

In [186]:
deepcheck_detectors.deepcheck_detect_drift(data_train=wine_dataset_train,
                                           data_to_compare=wine_dataset_corrupted_dioxide,
                                           label_col="quality",
                                           cat_features=categorical_features_names,
                                           model=lgbm_model,
                                           test_type="feature_drift")

Warning total sulfur dioxide


In [187]:
deepcheck_detectors.deepcheck_detect_drift(data_train=wine_dataset_train,
                                           data_to_compare=wine_dataset_corrupted_dioxide,
                                           label_col="quality",
                                           cat_features=categorical_features_names,
                                           model=lgbm_model,
                                           test_type="prediction_drift")

Warning


In [ ]:
#deepcheck_detectors.deepcheck_detect_drift(data_train=wine_dataset_train,
#                                           data_to_compare=wine_dataset_corrupted_dioxide,
#                                           label_col="quality",
#                                           cat_features=categorical_features_names,
#                                           model=lgbm_model,
#                                           test_type="dataset_drift")

### Evidently AI


In [188]:
evidently_ai_detectors.evidently_ai_detect_drift(data_train=wine_dataset_train,
                                                 data_to_compare=wine_dataset_corrupted_dioxide,
                                                 label_col="quality",
                                                 cat_features=categorical_features_names)

Alarm total sulfur dioxide


### Tensorflow


In [189]:
tensorflow_detectors.tensorflow_detect_drift(data_train=wine_dataset_train,
                                             data_to_compare=wine_dataset_corrupted_dioxide,
                                             label_col="quality")

,Anomaly short description,Anomaly long description
Feature name,,
'total sulfur dioxide',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 0.899405 (up to six significant digits), above the threshold 0.01."
'residual sugar',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 0.0556679 (up to six significant digits), above the threshold 0.01."
'sulphates',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 0.0382172 (up to six significant digits), above the threshold 0.01."
'density',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 0.0690513 (up to six significant digits), above the threshold 0.01."
'fixed acidity',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 0.026148 (up to six significant digits), above the threshold 0.01."
'alcohol',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 0.0151935 (up to six significant digits), above the threshold 0.01."
'chlorides',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 0.0295839 (up to six significant digits), above the threshold 0.01."
'free sulfur dioxide',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 0.0247023 (up to six significant digits), above the threshold 0.01."
'pH',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 0.0249625 (up to six significant digits), above the threshold 0.01."


### Drift Detectors with Labels

In [190]:
data_wine_dioxide = pd.concat([wine_dataset_train, wine_dataset_corrupted_dioxide], axis=0)


#### EDDM


In [191]:
drift_detector_with_labels.drift_detector_with_labels_test(data_to_compare=data_wine_dioxide,
                                                           label_col="quality",
                                                           model=lgbm_model,
                                                           test_name="EDDM")

Change detected at index 4109
Change detected at index 4262
Warning detected at index 4461
Change detected at index 6492


#### ADWIN


In [192]:
drift_detector_with_labels.drift_detector_with_labels_test(data_to_compare=data_wine_dioxide,
                                                           label_col="quality",
                                                           model=lgbm_model,
                                                           test_name="ADWIN")

Change detected at index 4063
Change detected at index 4095
Change detected at index 4159
Change detected at index 4191
Change detected at index 4223
Change detected at index 4479
Change detected at index 5119


#### HDDM_W


In [193]:
drift_detector_with_labels.drift_detector_with_labels_test(data_to_compare=data_wine_dioxide,
                                                           label_col="quality",
                                                           model=lgbm_model,
                                                           test_name="HDDM_W")

Warning detected at index 5965


### Drift Detector Multivariate: HDDDM & MD3


#### HDDDM


In [194]:
X_train = wine_dataset_train.loc[:, wine_dataset_train.columns != "quality"]
X_corrupted = wine_dataset_corrupted_dioxide.loc[:, wine_dataset_corrupted_dioxide.columns != "quality"]
drift_detector_multivariate_hdddm.hdddm_detect_drift(data_train=X_train,
                                                     data_to_compare=X_corrupted,
                                                     gamma_level=0.05)

None


#### MD3


In [195]:
drift_detector_multivariate_md3.md3_detect_drift(data_train=wine_dataset_train,
                                                 data_to_compare=wine_dataset_corrupted_dioxide,
                                                 label_col="quality")

Alarm, drift Detected


#### OLINDDA

In [196]:
X_corrupted = X_corrupted.reset_index(drop=True)

In [197]:
drift_detector_multivariate_ollindda.olindda_detect_drift(x_train_data=X_train,
                                                          x_test_data=X_corrupted,
                                                          n_clusters=6)

Novelty identified for clusters 1


True

## Virtual Concept Drift: swap values of the wine_type column

### Induce the drift

In [198]:
wine_dataset_corrupted_wine_type = drift.drift_generator_univariate_categorical_change(data=wine_dataset_test,
                                                                                       column_name='wine_type',
                                                                                       value1="1", value2="0")

### Deepcheck


In [199]:
deepcheck_detectors.deepcheck_detect_drift(data_train=wine_dataset_train,
                                           data_to_compare=wine_dataset_corrupted_wine_type,
                                           label_col="quality",
                                           cat_features=categorical_features_names,
                                           model=lgbm_model,
                                           test_type="feature_drift")

Alarm wine_type


In [200]:
wine_dataset_corrupted_wine_type['wine_type'] = wine_dataset_corrupted_wine_type['wine_type'].astype(int)
deepcheck_detectors.deepcheck_detect_drift(data_train=wine_dataset_train,
                                           data_to_compare=wine_dataset_corrupted_wine_type,
                                           label_col="quality",
                                           cat_features=categorical_features_names,
                                           model=lgbm_model,
                                           test_type="prediction_drift")

No drift detected


In [ ]:
#deepcheck_detectors.deepcheck_detect_drift(data_train=wine_dataset_train,
#                                           data_to_compare=wine_dataset_corrupted_wine_type,
#                                           label_col="quality",
#                                           cat_features=categorical_features_names,
#                                           model=lgbm_model,
#                                           test_type="dataset_drift")

### Evidently AI

In [201]:
evidently_ai_detectors.evidently_ai_detect_drift(data_train=wine_dataset_train,
                                                 data_to_compare=wine_dataset_corrupted_wine_type,
                                                 label_col="quality",
                                                 cat_features=categorical_features_names)

Alarm wine_type


### Tensorflow


In [202]:
tensorflow_detectors.tensorflow_detect_drift(data_train=wine_dataset_train,
                                             data_to_compare=wine_dataset_corrupted_wine_type,
                                             label_col="quality")

,Anomaly short description,Anomaly long description
Feature name,,
'wine_type',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 0.195302 (up to six significant digits), above the threshold 0.01."
'pH',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 0.0249625 (up to six significant digits), above the threshold 0.01."
'total sulfur dioxide',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 0.0127563 (up to six significant digits), above the threshold 0.01."
'fixed acidity',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 0.026148 (up to six significant digits), above the threshold 0.01."
'alcohol',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 0.0151935 (up to six significant digits), above the threshold 0.01."
'free sulfur dioxide',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 0.0247023 (up to six significant digits), above the threshold 0.01."
'citric acid',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 0.0356787 (up to six significant digits), above the threshold 0.01."
'sulphates',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 0.0382172 (up to six significant digits), above the threshold 0.01."
'volatile acidity',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 0.0318468 (up to six significant digits), above the threshold 0.01."


### Drift Detectors with Labels


In [203]:
data_wine_wine_type = pd.concat([wine_dataset_train, wine_dataset_corrupted_wine_type], axis=0)


#### EDDM


In [204]:
drift_detector_with_labels.drift_detector_with_labels_test(data_to_compare=data_wine_wine_type,
                                                           label_col="quality",
                                                           model=lgbm_model,
                                                           test_name="EDDM")

Change detected at index 4091
Change detected at index 4244
Change detected at index 4495
Change detected at index 4737
Change detected at index 5020
Change detected at index 5243
Change detected at index 5449
Change detected at index 5774
Change detected at index 6022
Warning detected at index 6275
Change detected at index 6337


#### ADWIN


In [205]:
drift_detector_with_labels.drift_detector_with_labels_test(data_to_compare=data_wine_wine_type,
                                                           label_col="quality",
                                                           model=lgbm_model,
                                                           test_name="ADWIN")

Change detected at index 4031
Change detected at index 4063
Change detected at index 4095
Change detected at index 4127
Change detected at index 4159
Change detected at index 4191
Change detected at index 4223
Change detected at index 4479


#### HDDM_W


In [206]:
drift_detector_with_labels.drift_detector_with_labels_test(data_to_compare=data_wine_wine_type,
                                                           label_col="quality",
                                                           model=lgbm_model,
                                                           test_name="HDDM_W")

### Drift Detector Multivariate: HDDDM & MD3
#### HDDDM


In [207]:
X_train = wine_dataset_train.loc[:, wine_dataset_train.columns != "quality"]
X_corrupted = wine_dataset_corrupted_wine_type.loc[:, wine_dataset_corrupted_wine_type.columns != "quality"]
drift_detector_multivariate_hdddm.hdddm_detect_drift(data_train=X_train,
                                                     data_to_compare=X_corrupted,
                                                     gamma_level=0.05)

None


#### MD3

In [208]:
drift_detector_multivariate_md3.md3_detect_drift(data_train=wine_dataset_train,
                                                 data_to_compare=wine_dataset_corrupted_wine_type,
                                                 label_col="quality")

Alarm, drift Detected


#### OLINDDA

In [209]:
drift_detector_multivariate_ollindda.olindda_detect_drift(x_train_data=X_train,
                                                          x_test_data=X_corrupted,
                                                          n_clusters=6)

No drift or novelty detected in batch


False

## Gradual Virtual Concept Drift: increase sulphates values by 0.1


### Deepcheck

In [210]:
deepcheck_detectors.deepcheck_detect_gradual_drift(data_train=wine_dataset_train,
                                                   data_to_compare=wine_dataset_test,
                                                   label_col="quality",
                                                   cat_features=categorical_features_names,
                                                   model=lgbm_model,
                                                   action="increase",
                                                   value_drift=0.1,
                                                   column_name="sulphates",
                                                   test_type="feature_drift",
                                                   nb_sample=100,
                                                   nb_days=5)

Day 0 :
Warning sulphates
Day 1 :
Alarm sulphates
Day 2 :
Alarm sulphates
Day 3 :
Alarm sulphates
Day 4 :
Alarm sulphates
Day 5 :
Alarm sulphates


In [211]:
deepcheck_detectors.deepcheck_detect_gradual_drift(data_train=wine_dataset_train,
                                                   data_to_compare=wine_dataset_test,
                                                   label_col="quality",
                                                   cat_features=categorical_features_names,
                                                   model=lgbm_model,
                                                   action="increase",
                                                   value_drift=0.1,
                                                   column_name="sulphates",
                                                   test_type="prediction_drift",
                                                   nb_sample=100,
                                                   nb_days=5)

Day 0 :
No drift detected
Day 1 :
No drift detected
Day 2 :
No drift detected
Day 3 :
No drift detected
Day 4 :
No drift detected
Day 5 :
No drift detected


In [ ]:
#deepcheck_detectors.deepcheck_detect_gradual_drift(data_train=wine_dataset_train,
#                                                  data_to_compare=wine_dataset_test,
#                                                   label_col="quality",
#                                                   cat_features=categorical_features_names,
#                                                   model=lgbm_model,
#                                                   action="increase",
#                                                   value_drift=0.1,
#                                                   column_name="sulphates",
#                                                   test_type="dataset_drift",
#                                                   nb_sample=100,
#                                                   nb_days=5)

### Evidently AI


In [212]:
evidently_ai_detectors.evidently_ai_detect_gradual_drift(data_train=wine_dataset_train,
                                                         data_to_compare=wine_dataset_test,
                                                         label_col="quality",
                                                         cat_features=categorical_features_names,
                                                         action="increase",
                                                         value_drift=0.1,
                                                         column_name="sulphates",
                                                         nb_sample=100,
                                                         nb_days=5)

Day 0 :
Alarm fixed acidity
Alarm citric acid
Alarm chlorides
Alarm pH
Alarm sulphates
Alarm alcohol
Day 1 :
Alarm fixed acidity
Alarm volatile acidity
Alarm citric acid
Alarm residual sugar
Alarm chlorides
Alarm free sulfur dioxide
Alarm total sulfur dioxide
Alarm density
Alarm pH
Alarm sulphates
Alarm alcohol
Day 2 :
Alarm citric acid
Alarm chlorides
Alarm sulphates
Alarm alcohol
Day 3 :
Alarm citric acid
Alarm residual sugar
Alarm chlorides
Alarm free sulfur dioxide
Alarm total sulfur dioxide
Alarm density
Alarm sulphates
Alarm alcohol
Day 4 :
Alarm fixed acidity
Alarm volatile acidity
Alarm citric acid
Alarm chlorides
Alarm total sulfur dioxide
Alarm pH
Alarm sulphates
Alarm alcohol
Day 5 :
Alarm fixed acidity
Alarm citric acid
Alarm total sulfur dioxide
Alarm pH
Alarm sulphates


### Tensorflow


In [213]:
tensorflow_detectors.tensorflow_detect_gradual_drift(data_train=wine_dataset_train,
                                                     data_to_compare=wine_dataset_test,
                                                     label_col="quality",
                                                     value_drift=0.1,
                                                     column_name="sulphates",
                                                     nb_sample=100,
                                                     nb_days=5)

Day 0 :


,Anomaly short description,Anomaly long description
Feature name,,
'chlorides',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 0.186419 (up to six significant digits), above the threshold 0.01."
'sulphates',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 1 (up to six significant digits), above the threshold 0.01."
'density',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 0.122935 (up to six significant digits), above the threshold 0.01."
'residual sugar',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 0.15975 (up to six significant digits), above the threshold 0.01."
'total sulfur dioxide',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 0.0271687 (up to six significant digits), above the threshold 0.01."
'fixed acidity',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 0.0660109 (up to six significant digits), above the threshold 0.01."
'alcohol',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 0.0541923 (up to six significant digits), above the threshold 0.01."
'free sulfur dioxide',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 0.0343012 (up to six significant digits), above the threshold 0.01."
'citric acid',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 0.0520243 (up to six significant digits), above the threshold 0.01."


Day 1 :


,Anomaly short description,Anomaly long description
Feature name,,
'density',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 0.0750489 (up to six significant digits), above the threshold 0.01."
'chlorides',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 0.147667 (up to six significant digits), above the threshold 0.01."
'alcohol',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 0.0469872 (up to six significant digits), above the threshold 0.01."
'citric acid',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 0.0343648 (up to six significant digits), above the threshold 0.01."
'sulphates',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 1 (up to six significant digits), above the threshold 0.01."
'free sulfur dioxide',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 0.0491078 (up to six significant digits), above the threshold 0.01."
'pH',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 0.0289179 (up to six significant digits), above the threshold 0.01."
'volatile acidity',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 0.0662771 (up to six significant digits), above the threshold 0.01."
'fixed acidity',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 0.0473311 (up to six significant digits), above the threshold 0.01."


Day 2 :


...

,Anomaly short description,Anomaly long description
Feature name,,
'sulphates',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 0.767 (up to six significant digits), above the threshold 0.01."
'chlorides',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 0.163016 (up to six significant digits), above the threshold 0.01."
'pH',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 0.0576444 (up to six significant digits), above the threshold 0.01."
'alcohol',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 0.0454991 (up to six significant digits), above the threshold 0.01."
'free sulfur dioxide',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 0.104724 (up to six significant digits), above the threshold 0.01."
'total sulfur dioxide',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 0.0341484 (up to six significant digits), above the threshold 0.01."
'citric acid',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 0.0386768 (up to six significant digits), above the threshold 0.01."
'fixed acidity',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 0.0785898 (up to six significant digits), above the threshold 0.01."
'residual sugar',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 0.147112 (up to six significant digits), above the threshold 0.01."


Day 4 :


,Anomaly short description,Anomaly long description
Feature name,,
'chlorides',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 0.128416 (up to six significant digits), above the threshold 0.01."
'sulphates',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 0.658427 (up to six significant digits), above the threshold 0.01."
'density',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 0.116058 (up to six significant digits), above the threshold 0.01."
'residual sugar',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 0.099558 (up to six significant digits), above the threshold 0.01."
'total sulfur dioxide',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 0.0217083 (up to six significant digits), above the threshold 0.01."
'fixed acidity',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 0.050184 (up to six significant digits), above the threshold 0.01."
'alcohol',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 0.0289192 (up to six significant digits), above the threshold 0.01."
'free sulfur dioxide',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 0.0587585 (up to six significant digits), above the threshold 0.01."
'citric acid',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 0.0492725 (up to six significant digits), above the threshold 0.01."


Day 5 :


,Anomaly short description,Anomaly long description
Feature name,,
'volatile acidity',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 0.112435 (up to six significant digits), above the threshold 0.01."
'citric acid',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 0.0499088 (up to six significant digits), above the threshold 0.01."
'residual sugar',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 0.0995603 (up to six significant digits), above the threshold 0.01."
'chlorides',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 0.117103 (up to six significant digits), above the threshold 0.01."
'alcohol',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 0.0396991 (up to six significant digits), above the threshold 0.01."
'density',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 0.121429 (up to six significant digits), above the threshold 0.01."
'pH',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 0.0551317 (up to six significant digits), above the threshold 0.01."
'fixed acidity',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 0.0541513 (up to six significant digits), above the threshold 0.01."
'total sulfur dioxide',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 0.0280939 (up to six significant digits), above the threshold 0.01."


### Drift Detector with Labels


#### EDDM

In [214]:
drift_detector_with_labels.drift_detector_labels_gradual_drift(data_train=wine_dataset_train,
                                                               data_to_compare=wine_dataset_test,
                                                               column_name="sulphates",
                                                               label_col="quality",
                                                               model=lgbm_model,
                                                               value_drift=0.1,
                                                               action="increase",
                                                               test_name="EDDM",
                                                               nb_sample=100,
                                                               nb_days=5)

Day 0 :
Day 1 :
Day 2 :
Day 3 :
Day 4 :
Day 5 :


#### ADWIN

In [215]:
drift_detector_with_labels.drift_detector_labels_gradual_drift(data_train=wine_dataset_train,
                                                               data_to_compare=wine_dataset_test,
                                                               column_name="sulphates",
                                                               label_col="quality",
                                                               model=lgbm_model,
                                                               value_drift=0.1,
                                                               action="increase",
                                                               test_name="EDDM",
                                                               nb_sample=100,
                                                               nb_days=5)

Day 0 :
Day 1 :
Day 2 :
Day 3 :
Day 4 :
Day 5 :


#### HDDM_W

In [216]:
drift_detector_with_labels.drift_detector_labels_gradual_drift(data_train=wine_dataset_train,
                                                               data_to_compare=wine_dataset_test,
                                                               column_name="sulphates",
                                                               label_col="quality",
                                                               model=lgbm_model,
                                                               value_drift=0.1,
                                                               action="increase",
                                                               test_name="EDDM",
                                                               nb_sample=100,
                                                               nb_days=5)

Day 0 :
Day 1 :
Day 2 :
Day 3 :
Day 4 :
Day 5 :


### Drift Detector Multivariate: HDDDM & MD3
#### HDDDM

In [217]:
X_train = wine_dataset_train.loc[:, wine_dataset_train.columns != "quality"]
X_corrupted = wine_dataset_test.loc[:, wine_dataset_test.columns != "quality"]

In [218]:
drift_detector_multivariate_hdddm.hdddm_detect_gradual_drift(data_train=X_train,
                                                             data_to_compare=X_corrupted,
                                                             column_name="sulphates",
                                                             value_drift=0.1,
                                                             action="increase",
                                                             nb_sample=100,
                                                             nb_days=5)

Day 0 :
None
Day 1 :
None
Day 2 :
None
Day 3 :
None
Day 4 :
None
Day 5 :
None


#### MD3

In [219]:
drift_detector_multivariate_md3.md3_gradual_drift(data_train=wine_dataset_train,
                                                  data_to_compare=wine_dataset_test,
                                                  label_col="quality",
                                                  column_name="sulphates",
                                                  value_drift=0.1,
                                                  action="increase",
                                                  nb_sample=100,
                                                  nb_days=5)

Day 0 :
Alarm, drift Detected
Day 1 :
Alarm, drift Detected
Day 2 :
Day 3 :
Alarm, drift Detected
Day 4 :
Alarm, drift Detected
Day 5 :
Alarm, drift Detected


#### OLINDDA

In [220]:
drift_detector_multivariate_ollindda.olindda_gradual_drift(data_train=X_train,
                                                             data_to_compare=X_corrupted,
                                                           n_clusters=6,
                                                             column_name="sulphates",
                                                             value_drift=0.1,
                                                             action="increase",
                                                             nb_sample=100,
                                                             nb_days=5)

Day 0 :
No drift or novelty detected in batch
Day 1 :
No drift or novelty detected in batch
Day 2 :
No drift or novelty detected in batch
Day 3 :
No drift or novelty detected in batch
Day 4 :
No drift or novelty detected in batch
Day 5 :
No drift or novelty detected in batch


## Seasonal Virtual Concept Drift: increase pH by 50% every two days

### Deepcheck


In [221]:
deepcheck_detectors.deepcheck_detect_seasonal_drift(data_train=wine_dataset_train,
                                                   data_to_compare=wine_dataset_test,
                                                   label_col="quality",
                                                   cat_features=categorical_features_names,
                                                   model=lgbm_model,
                                                   action="multiply",
                                                   value_drift=1.5,
                                                   column_name="pH",
                                                   test_type="feature_drift",
                                                   nb_sample=100,
                                                    frequency=3,
                                                   nb_days=10)

Day 0 :
Alarm pH
Day 1 :
Day 2 :
Day 3 :
Alarm pH
Day 4 :
Day 5 :
Day 6 :
Alarm pH
Day 7 :
Day 8 :
Day 9 :
Alarm pH
Day 10 :


In [222]:
deepcheck_detectors.deepcheck_detect_seasonal_drift(data_train=wine_dataset_train,
                                                   data_to_compare=wine_dataset_test,
                                                   label_col="quality",
                                                   cat_features=categorical_features_names,
                                                   model=lgbm_model,
                                                   action="multiply",
                                                   value_drift=1.5,
                                                   column_name="pH",
                                                   test_type="prediction_drift",
                                                   nb_sample=100,
                                                    frequency=3,
                                                   nb_days=10)

Day 0 :
No drift detected
Day 1 :
No drift detected
Day 2 :
No drift detected
Day 3 :
No drift detected
Day 4 :
No drift detected
Day 5 :
No drift detected
Day 6 :
No drift detected
Day 7 :
No drift detected
Day 8 :
No drift detected
Day 9 :
No drift detected
Day 10 :
No drift detected


In [ ]:
#deepcheck_detectors.deepcheck_detect_seasonal_drift(data_train=wine_dataset_train,
#                                                   data_to_compare=wine_dataset_test,
#                                                   label_col="quality",
#                                                   cat_features=categorical_features_names,
#                                                   model=lgbm_model,
#                                                   action="multiply",
#                                                   value_drift=1.5,
#                                                   column_name="pH",
#                                                   test_type="dataset_drift",
#                                                   nb_sample=100,
#                                                    frequency=3,
#                                                   nb_days=10)

### Evidently AI


In [223]:
evidently_ai_detectors.evidently_ai_detect_seasonal_drift(data_train=wine_dataset_train,
                                                         data_to_compare=wine_dataset_test,
                                                         label_col="quality",
                                                         cat_features=categorical_features_names,
                                                         action="multiply",
                                                         value_drift=1.5,
                                                         column_name="pH",
                                                         frequency=3,
                                                         nb_sample=100,
                                                         nb_days=10)

Day 0 :
Alarm fixed acidity
Alarm citric acid
Alarm residual sugar
Alarm chlorides
Alarm free sulfur dioxide
Alarm total sulfur dioxide
Alarm density
Alarm pH
Alarm sulphates
Alarm alcohol
Day 1 :
Alarm fixed acidity
Alarm volatile acidity
Alarm citric acid
Alarm residual sugar
Alarm chlorides
Alarm free sulfur dioxide
Alarm total sulfur dioxide
Alarm density
Alarm pH
Alarm sulphates
Alarm alcohol
Day 2 :
Alarm fixed acidity
Alarm volatile acidity
Alarm citric acid
Alarm chlorides
Alarm pH
Alarm sulphates
Alarm alcohol
Alarm quality
Day 3 :
Alarm fixed acidity
Alarm volatile acidity
Alarm citric acid
Alarm chlorides
Alarm free sulfur dioxide
Alarm total sulfur dioxide
Alarm pH
Alarm sulphates
Day 4 :
Alarm citric acid
Alarm chlorides
Alarm pH
Alarm sulphates
Day 5 :
Alarm volatile acidity
Alarm chlorides
Alarm free sulfur dioxide
Alarm alcohol
Day 6 :
Alarm fixed acidity
Alarm volatile acidity
Alarm free sulfur dioxide
Alarm pH
Alarm alcohol
Day 7 :
Alarm citric acid
Alarm free sulfur 

### Tensorflow

In [224]:
tensorflow_detectors.tensorflow_detect_seasonal_drift(data_train=wine_dataset_train,
                                                     data_to_compare=wine_dataset_test,
                                                     label_col="quality",
                                                     value_drift=1.5,
                                                     column_name="pH",
                                                      action="multiply",
                                                     frequency=3,
                                                     nb_sample=100,
                                                     nb_days=10)

Day 0 :


,Anomaly short description,Anomaly long description
Feature name,,
'chlorides',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 0.105419 (up to six significant digits), above the threshold 0.01."
'total sulfur dioxide',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 0.0408722 (up to six significant digits), above the threshold 0.01."
'volatile acidity',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 0.0521241 (up to six significant digits), above the threshold 0.01."
'free sulfur dioxide',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 0.0407245 (up to six significant digits), above the threshold 0.01."
'fixed acidity',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 0.0493097 (up to six significant digits), above the threshold 0.01."
'pH',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 1 (up to six significant digits), above the threshold 0.01."
'density',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 0.129452 (up to six significant digits), above the threshold 0.01."
'alcohol',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 0.0404995 (up to six significant digits), above the threshold 0.01."
'residual sugar',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 0.107767 (up to six significant digits), above the threshold 0.01."


Day 1 :


,Anomaly short description,Anomaly long description
Feature name,,
'volatile acidity',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 0.0866928 (up to six significant digits), above the threshold 0.01."
'fixed acidity',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 0.0750086 (up to six significant digits), above the threshold 0.01."
'residual sugar',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 0.0847827 (up to six significant digits), above the threshold 0.01."
'density',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 0.10768 (up to six significant digits), above the threshold 0.01."
'sulphates',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 0.0685906 (up to six significant digits), above the threshold 0.01."
'chlorides',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 0.107076 (up to six significant digits), above the threshold 0.01."
'pH',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 0.0350716 (up to six significant digits), above the threshold 0.01."
'alcohol',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 0.0405337 (up to six significant digits), above the threshold 0.01."
'total sulfur dioxide',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 0.0386404 (up to six significant digits), above the threshold 0.01."


Day 2 :


...

,Anomaly short description,Anomaly long description
Feature name,,
'alcohol',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 0.0256879 (up to six significant digits), above the threshold 0.01."
'volatile acidity',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 0.0804303 (up to six significant digits), above the threshold 0.01."
'pH',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 0.0500445 (up to six significant digits), above the threshold 0.01."
'citric acid',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 0.0697686 (up to six significant digits), above the threshold 0.01."
'residual sugar',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 0.130599 (up to six significant digits), above the threshold 0.01."
'total sulfur dioxide',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 0.0420976 (up to six significant digits), above the threshold 0.01."
'chlorides',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 0.15624 (up to six significant digits), above the threshold 0.01."
'density',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 0.137453 (up to six significant digits), above the threshold 0.01."
'sulphates',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 0.0809762 (up to six significant digits), above the threshold 0.01."


Day 9 :


,Anomaly short description,Anomaly long description
Feature name,,
'pH',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 1 (up to six significant digits), above the threshold 0.01."
'total sulfur dioxide',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 0.0505399 (up to six significant digits), above the threshold 0.01."
'citric acid',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 0.047133 (up to six significant digits), above the threshold 0.01."
'density',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 0.133966 (up to six significant digits), above the threshold 0.01."
'chlorides',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 0.130124 (up to six significant digits), above the threshold 0.01."
'free sulfur dioxide',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 0.0811279 (up to six significant digits), above the threshold 0.01."
'fixed acidity',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 0.0536629 (up to six significant digits), above the threshold 0.01."
'sulphates',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 0.0803298 (up to six significant digits), above the threshold 0.01."
'alcohol',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 0.0451772 (up to six significant digits), above the threshold 0.01."


Day 10 :


,Anomaly short description,Anomaly long description
Feature name,,
'alcohol',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 0.0743059 (up to six significant digits), above the threshold 0.01."
'free sulfur dioxide',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 0.0510628 (up to six significant digits), above the threshold 0.01."
'citric acid',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 0.0585808 (up to six significant digits), above the threshold 0.01."
'fixed acidity',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 0.0353849 (up to six significant digits), above the threshold 0.01."
'sulphates',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 0.109554 (up to six significant digits), above the threshold 0.01."
'residual sugar',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 0.122924 (up to six significant digits), above the threshold 0.01."
'volatile acidity',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 0.0750442 (up to six significant digits), above the threshold 0.01."
'pH',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 0.0401817 (up to six significant digits), above the threshold 0.01."
'density',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 0.0713843 (up to six significant digits), above the threshold 0.01."


### Drift Detector with Labels
#### EDDM

In [231]:
drift_detector_with_labels.drift_detector_labels_seasonal_drift(data_train=wine_dataset_train,
                                                               data_to_compare=wine_dataset_test,
                                                               column_name="pH",
                                                               label_col="quality",
                                                               model=lgbm_model,
                                                               frequency=3,
                                                               value_drift=1.5,
                                                               action="multiply",
                                                               test_name="EDDM",
                                                               nb_sample=100,
                                                               nb_days=10)

Day 0 :
Day 1 :
Day 2 :
Day 3 :
Day 4 :
Day 5 :
Day 6 :
Day 7 :
Day 8 :
Day 9 :
Day 10 :


In [232]:
drift_detector_with_labels.drift_detector_labels_seasonal_drift(data_train=wine_dataset_train,
                                                               data_to_compare=wine_dataset_test,
                                                               column_name="pH",
                                                               label_col="quality",
                                                               model=lgbm_model,
                                                               frequency=3,
                                                               value_drift=1.5,
                                                               action="multiply",
                                                               test_name="HDDM_W",
                                                               nb_sample=100,
                                                               nb_days=10)

Day 0 :
Day 1 :
Day 2 :
Day 3 :
Day 4 :
Day 5 :
Day 6 :
Day 7 :
Day 8 :
Day 9 :
Day 10 :


In [233]:
drift_detector_with_labels.drift_detector_labels_seasonal_drift(data_train=wine_dataset_train,
                                                               data_to_compare=wine_dataset_test,
                                                               column_name="pH",
                                                               label_col="quality",
                                                               model=lgbm_model,
                                                               frequency=3,
                                                               value_drift=1.5,
                                                               action="multiply",
                                                               test_name="ADWIN",
                                                               nb_sample=100,
                                                               nb_days=10)

Day 0 :
Day 1 :
Day 2 :
Day 3 :
Day 4 :
Day 5 :
Day 6 :
Day 7 :
Day 8 :
Day 9 :
Day 10 :


### Drift Detector Multivariate: HDDDM & MD3
#### HDDDM

In [228]:
X_train = wine_dataset_train.loc[:, wine_dataset_train.columns != "quality"]
X_corrupted = wine_dataset_test.loc[:, wine_dataset_test.columns != "quality"]
drift_detector_multivariate_hdddm.hdddm_detect_seasonal_drift(data_train=X_train,
                                                             data_to_compare=X_corrupted,
                                                             column_name="pH",
                                                             value_drift=1.5,
                                                             action="multiply",
                                                             nb_sample=100,
                                                             frequency=3,
                                                             nb_days=10)

Day 0 :
None
Day 1 :
None
Day 2 :
None
Day 3 :
drift
Day 4 :
None
Day 5 :
None
Day 6 :
None
Day 7 :
None
Day 8 :
drift
Day 9 :
None
Day 10 :
None


#### MD3

In [229]:
drift_detector_multivariate_md3.md3_seasonal_drift(data_train=wine_dataset_train,
                                                   data_to_compare=wine_dataset_test,
                                                   label_col="quality",
                                                   column_name="pH",
                                                   value_drift=1.5,
                                                   action="multiply",
                                                   nb_sample=100,
                                                   frequency=3,
                                                   nb_days=10)

Day 0 :
Alarm, drift Detected
Day 1 :
Alarm, drift Detected
Day 2 :
Alarm, drift Detected
Day 3 :
Alarm, drift Detected
Day 4 :
Alarm, drift Detected
Day 5 :
Day 6 :
Alarm, drift Detected
Day 7 :
Alarm, drift Detected
Day 8 :
Alarm, drift Detected
Day 9 :
Alarm, drift Detected
Day 10 :
Alarm, drift Detected


#### OLINDDA

In [230]:
drift_detector_multivariate_ollindda.olindda_seasonal_drift(data_train=X_train,
                                                   data_to_compare=X_corrupted,
                                                            n_clusters=6,
                                                   column_name="pH",
                                                   value_drift=1.5,
                                                   action="multiply",
                                                   nb_sample=100,
                                                   frequency=3,
                                                   nb_days=10)

Day 0 :
No drift or novelty detected in batch
Day 1 :
No drift or novelty detected in batch
Day 2 :
No drift or novelty detected in batch
Day 3 :
No drift or novelty detected in batch
Day 4 :
No drift or novelty detected in batch
Day 5 :
No drift or novelty detected in batch
Day 6 :
No drift or novelty detected in batch
Day 7 :
No drift or novelty detected in batch
Day 8 :
No drift or novelty detected in batch
Day 9 :
No drift or novelty detected in batch
Day 10 :
No drift or novelty detected in batch
